In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import seaborn as sns

FILTERED_EIDS = [
    '56b57c38-2699-4091-90a8-aba35103155e',
    '7af49c00-63dd-4fed-b2e0-1b3bd945b20b',
    '862ade13-53cd-4221-a3fa-dda8643641f2'
]

In [2]:
def compute_ensemble_stddev(
    df_preds,
    keypoint_ensemble_list,
    scorer_name='standard_scorer'
):
    """
    Parameters
    ----------
    df_ground_truth : List[pd.DataFrame]
        ground truth predictions
    df_preds : List[pd.DataFrame]
        model predictions
    keypoint_ensemble_list : List[str]
        keypoints to include in the analysis

    Returns
    -------
    np.ndarray
        shape (n_frames, n_keypoints)
    """
    # Initial check for NaNs in df_preds
    for i, df in enumerate(df_preds):
        if df.isna().any().any():
            print(f"Warning: NaN values detected in initial DataFrame {i}.")
            nan_indices = df[df.isna().any(axis=1)].index
            nan_columns = df.columns[df.isna().any()]
            print(f"NaN values found at indices: {nan_indices} in columns: {nan_columns}")

    preds = []
    cols_order = None
    for i, df in enumerate(df_preds):

        # Standardize the 'scorer' level
        df = standardize_scorer_level(df, scorer_name)

        # Remove likelihood columns
        cols_to_keep = [
            col for col in df.columns
            if not col[2].endswith('likelihood') and 'zscore' not in col[2]]
        # Keep only columns matching the keypoint_ensemble_list
        cols_to_keep = [col for col in cols_to_keep if col[1] in keypoint_ensemble_list]
        df = df[cols_to_keep]

        # print(f"DataFrame {i} kept columns:", df.columns)

        # Check for NaNs in the DataFrame
        if df.isna().any().any():
            print(f"Warning: NaN values detected in DataFrame {i} after filtering.")
            nan_indices = df[df.isna().any(axis=1)].index
            nan_columns = df.columns[df.isna().any()]
            print(f"NaN values found at indices: {nan_indices} in columns: {nan_columns}")

        # Print the order of the column headers
        if cols_order is None:
            cols_order = df.columns
        else:
            if not (df.columns == cols_order).all():
                print(f"Column order mismatch detected in DataFrame {i}")
                print("Expected order:", cols_order)
                print("Actual order:", df.columns)
                # Ensure bodyparts and coordinates are consistent
                expected_bodyparts_coords = cols_order.droplevel(0).unique()
                actual_bodyparts_coords = df.columns.droplevel(0).unique()
                if not expected_bodyparts_coords.equals(actual_bodyparts_coords):
                    print("Bodyparts and coordinates mismatch detected")
                    print("Expected bodyparts and coordinates:", expected_bodyparts_coords)
                    print("Actual bodyparts and coordinates:", actual_bodyparts_coords)

        # Reshape the DataFrame to the appropriate shape
        try:
            arr = df.to_numpy().reshape(df.shape[0], -1, 2)
        except ValueError as e:
            print(f"Reshape error: {e}")
            print(f"DataFrame shape: {df.shape}")
            print(f"Array shape after reshape attempt: {df.to_numpy().shape}")
            raise

        preds.append(arr[..., None])

    preds = np.concatenate(preds, axis=3)

    # Check for NaNs in preds
    if np.isnan(preds).any():
        print("Warning: NaN values detected in preds array.")
        nan_indices = np.argwhere(np.isnan(preds))
        print(f"NaN values found at indices: {nan_indices}")
    else:
        print("No NaN values detected in preds array.")

    stddevs = np.std(preds, axis=-1).mean(axis=-1)
    # print(f"Stddevs: {stddevs}")
    return stddevs


def standardize_scorer_level(df, new_scorer='standard_scorer'):
    """
    Standardizes the 'scorer' level in the MultiIndex to a common name.

    Parameters
    ----------
    df : pd.DataFrame
        The DataFrame to standardize.
    new_scorer : str
        The new name for the 'scorer' level.

    Returns
    -------
    pd.DataFrame
        The DataFrame with the standardized 'scorer' level.
    """
    df.columns = pd.MultiIndex.from_tuples(
        [(new_scorer, bodypart, coord) for scorer, bodypart, coord in df.columns],
        names=df.columns.names
    )
    return df


def compute_percentiles(arr, std_vals, percentiles):
    num_pts = arr[0]
    vals = []
    prctiles = []
    for p in percentiles:
        v = num_pts * p / 100
        idx = np.argmin(np.abs(arr - v))
        # maybe we don't have enough data
        if idx == len(arr) - 1:
            p_ = arr[idx] / num_pts * 100
        else:
            p_ = p
        vals.append(std_vals[idx])
        prctiles.append(np.round(p_, 2))
    return vals, prctiles


def compute_ensemble_stddev(
    df_preds,
    scorer_name="standard_scorer",
):
    """
    Parameters
    ----------
    df_preds : List[pd.DataFrame]
        model predictions

    Returns
    -------
    np.ndarray
        shape (n_frames, n_keypoints)
    """
    # Initial check for NaNs in df_preds
    for i, df in enumerate(df_preds):
        if df.isna().any().any():
            print(f"Warning: NaN values detected in initial DataFrame {i}.")
            nan_indices = df[df.isna().any(axis=1)].index
            nan_columns = df.columns[df.isna().any()]
            print(
                f"NaN values found at indices: {nan_indices} in columns: {nan_columns}"
            )

    preds = []
    for i, df in enumerate(df_preds):
        # Standardize the 'scorer' level
        df = standardize_scorer_level(df, scorer_name)

        # Remove likelihood columns
        cols_to_keep = [
            col
            for col in df.columns
            if not col[2].endswith("likelihood") and "Unnamed" not in col[2]
        ]
        df = df[cols_to_keep]

        # print(f"DataFrame {i} kept columns:", df.columns)

        # Check for NaNs in the DataFrame
        if df.isna().any().any():
            print(f"Warning: NaN values detected in DataFrame {i} after filtering.")
            nan_indices = df[df.isna().any(axis=1)].index
            nan_columns = df.columns[df.isna().any()]
            print(
                f"NaN values found at indices: {nan_indices} in columns: {nan_columns}"
            )

        # Reshape the DataFrame to the appropriate shape
        try:
            arr = df.to_numpy().reshape(df.shape[0], -1, 2)
        except ValueError as e:
            print(f"Reshape error: {e}")
            print(f"DataFrame shape: {df.shape}")
            print(f"Array shape after reshape attempt: {df.to_numpy().shape}")
            raise

        preds.append(arr[..., None])

    preds = np.concatenate(preds, axis=3)

    # Check for NaNs in preds
    if np.isnan(preds).any():
        print("Warning: NaN values detected in preds array.")
        nan_indices = np.argwhere(np.isnan(preds))
        print(f"NaN values found at indices: {nan_indices}")
    else:
        print("No NaN values detected in preds array.")

    stddevs = np.std(preds, axis=-1).mean(axis=-1)
    # print(f"Stddevs: {stddevs}")
    return stddevs


def standardize_scorer_level(df, new_scorer="standard_scorer"):
    """
    Standardizes the 'scorer' level in the MultiIndex to a common name.

    Parameters
    ----------
    df : pd.DataFrame
        The DataFrame to standardize.
    new_scorer : str
        The new name for the 'scorer' level.

    Returns
    -------
    pd.DataFrame
        The DataFrame with the standardized 'scorer' level.
    """
    df.columns = pd.MultiIndex.from_tuples(
        [(new_scorer, bodypart, coord) for scorer, bodypart, coord in df.columns],
        names=df.columns.names,
    )
    return df


def compute_percentiles(arr, std_vals, percentiles):
    num_pts = arr[0]
    vals = []
    prctiles = []
    for p in percentiles:
        v = num_pts * p / 100
        idx = np.argmin(np.abs(arr - v))
        # maybe we don't have enough data
        if idx == len(arr) - 1:
            p_ = arr[idx] / num_pts * 100
        else:
            p_ = p
        vals.append(std_vals[idx])
        prctiles.append(np.round(p_, 2))
    return vals, prctiles


class Ensemble:
    data_to_plot: dict[str, str]  # Model name to path.
    pred_csv_list: list[str]
    error_csv_list: list[str]
    df_pred_list: list[pd.DataFrame]
    df_error_list: list[pd.DataFrame]
    n_points_dict: dict[str, dict[str, int]]  # Model name -> std_val_inx -> num points
    std_vals: list[float]
    df_w_vars: pd.DataFrame
    df_line2: pd.DataFrame

        
def build_ensemble(ens: Ensemble):
    data_to_plot = ens.data_to_plot
    # define predictions so we can compute ensemble variance
    pred_csv_list = []
    # define model names
    model_names_list = []
    # reformat
    for key, val in data_to_plot.items():
        model_names_list.append(key)
        pred_csv_list.append(Path(val))

    error_csv_list = []
    for p in pred_csv_list:
        p = p.with_stem(p.stem.replace("remapped_", ""))
        p = p.with_stem(p.stem.replace("_new", "") + "_pixel_error_new")
        error_csv_list.append(p)

    # load data
    df_pred_list = []
    df_error_list = []
    for pred_csv, error_csv in zip(pred_csv_list, error_csv_list):
        preds_df = pd.read_csv(pred_csv, header=[0, 1, 2], index_col=0).sort_index()
        # remove eids that are in FILTERED_EIDS
        # any FILTERED_EIDS in preds_df.index, remove the eids
        remove_indices = [i for i in preds_df.index if any(eid in i for eid in FILTERED_EIDS)]
        preds_df = preds_df.drop(remove_indices)
        df_pred_list.append(preds_df)
        df = pd.read_csv(error_csv, header=[0], index_col=0).sort_index()
        remove_indices = [i for i in df.index if any(eid in i for eid in FILTERED_EIDS)]
        df = df.drop(remove_indices)
        if "set" in df.columns:
            df = df.drop(columns=["set"])
        df_error_list.append(df)

    # compute ensemble variance
    ens_stddev = compute_ensemble_stddev(
        df_pred_list,
    )

    # Record pixel errors along with ensemble variances
    df_w_vars = []
    # Dataframe:
    #   Index = {img_path}_{model_name}_{kp}: str
    #   model: str
    #   keypoint: str
    #   pixel_error: float
    #   ens-std: float

    for df_error, df_pred, model_name in zip(
        df_error_list, df_pred_list, model_names_list
    ):
        # Ensure df_error has the same index as df_gt
        assert (df_error.index == df_pred_list[0].index).all()
        assert (df_pred.index == df_pred_list[0].index).all()

        # Calculate total pixel error (sum) for each df_error and print it
        total_pixel_error = df_error.sum().sum()
        print(f"Total pixel error for model {model_name}: {total_pixel_error}")

        for i, kp in enumerate(df_error.columns):
            index = [f"{i}_{model_name}_{kp}" for i in df_error.index]
            df_w_vars.append(
                pd.DataFrame(
                    {
                        "pixel_error": df_error[kp].values,
                        "likelihood": df_pred.loc[
                            :, ("standard_scorer", kp, "likelihood")
                        ].values,
                        "ens-std": ens_stddev[:, i],
                        "ens-std-prctile": [
                            np.sum(ens_stddev < p) / ens_stddev.size
                            for p in ens_stddev[:, i]
                        ],
                        "ens-std-prctile-kp": [
                            np.sum(ens_stddev[:, i] < p) / ens_stddev[:, i].size
                            for p in ens_stddev[:, i]
                        ],
                        "keypoint": kp,
                        "model": model_name,
                    },
                    index=index,
                )
            )

    df_w_vars = pd.concat(df_w_vars)

    std_vals = np.arange(0, 8, 0.2)

    # Dict from [model_name][std_vals_index] to the number of points for that model that were greater than
    # std_vals[std_vals_index].
    n_points_dict = {m: np.nan * np.zeros_like(std_vals) for m in model_names_list}

    # Like df_w_vars but one per std_val
    # Dataframe:
    #   Index = {img_path}_{model_name}_{kp}_{std_val_index}: str
    #   model: str
    #   keypoint: str
    #   pixel_error: float
    #   ens-std: float
    #   n_points: int
    df_line2 = []

    for s, std in enumerate(std_vals):
        # Get all
        df_tmp_ = df_w_vars[df_w_vars["ens-std"] > std]
        for model_name in model_names_list:
            d = df_tmp_[df_tmp_.model == model_name]
            n_points = np.sum(~d["pixel_error"].isna())
            n_points_dict[model_name][s] = n_points
            index = []
            for row, keypoint in zip(d.index, d["keypoint"].to_numpy()):
                index.append(f"{row}_{s}")

            df_line2.append(
                pd.DataFrame(
                    {
                        "ens-std": std,
                        "model": model_name,
                        "pixel_error": d.pixel_error.to_numpy(),
                        "keypoint": d["keypoint"].to_numpy(),
                    },
                    index=index,
                )
            )

    df_line2 = pd.concat(df_line2)
    ens.pred_csv_list = pred_csv_list
    ens.error_csv_list = error_csv_list
    ens.df_pred_list = df_pred_list
    ens.df_error_list = df_error_list
    ens.n_points_dict = n_points_dict
    ens.std_vals = std_vals
    ens.df_w_vars = df_w_vars
    ens.df_line2 = df_line2
    return ens

Parameters

In [ ]:
models = [
    'resnet50_animal_ap10k_type-sv', 
    'vitb_imagenet_type-sv', 
    'vitb-mae_type-sv',
    'vitb-mvt_type-sv',
    'vits-dino-mvt_type-sv',
    'vits_dino_type-sv',
    'vitb-mae_type-mv',
    'vitb-mae_type-mva',
    'vits_dino_type-mv',
    'vits_dino_type-mva',
    'vitb_dino_type-mv',
    'vitb_imagenet_type-mv',
    'vitb_imagenet_type-mva'
]
colors = sns.color_palette("husl", len(models))
# Define the color mapping
color_mapping = {k: colors[i] for i, k in enumerate(models)}

# Define litpose save dir
LITPOSE_SAVE_DIR = Path("../../outputs/litpose").resolve()
print(f"LITPOSE_SAVE_DIR: {LITPOSE_SAVE_DIR}")
print(f"Color mapping: {color_mapping}")

def get_litpose_save_dir(dataset, model, frame, seed, mode='ft'):
    if model.endswith('_type-sv'):
        model_type = 'heatmap'
    elif model.endswith('_type-mv'):
        model_type = 'heatmap_multiview_transformer'
    elif model.endswith('_type-mva'):
        model_type = 'heatmap_multiview_aggregator'
    else:
        raise ValueError(f"Unknown model: {model}")
    model_name = model.split('_type-')[0]
    return Path(f"{LITPOSE_SAVE_DIR}/{dataset}/ds-{dataset}_mode-{mode}_model-{model_name}_type-{model_type}_frame-{frame}_epoch-300_seed-{seed}").resolve()

LITPOSE_SAVE_DIR: /mnt/home/DDN_Copy/columbia/ywang1/Projects/multi-view/outputs/litpose
Color mapping: {'resnet50_animal_ap10k_type-sv': (0.9677975592919913, 0.44127456009157356, 0.5358103155058701), 'vitb_imagenet_type-sv': (0.9218528551477546, 0.5003454332640002, 0.1960624705339954), 'vitb-mae_type-sv': (0.756593795426372, 0.58660455674609, 0.19460548454202936), 'vitb-mvt_type-sv': (0.6280838378584804, 0.6317149736053096, 0.19371846323785552), 'vits-dino-mvt_type-sv': (0.44127702936426016, 0.6747986506725627, 0.19278677787286805), 'vits_dino_type-sv': (0.19925855442314272, 0.6935861010390585, 0.4349484671340832), 'vitb-mae_type-mv': (0.20730980534421067, 0.6820252887361699, 0.5931178995944008), 'vitb-mae_type-mva': (0.213603436221035, 0.6724447149771248, 0.6900417229996091), 'vits_dino_type-mv': (0.2218708221705632, 0.6590814541823624, 0.7990680761766646), 'vits_dino_type-mva': (0.38670894369284947, 0.6148978276981065, 0.9585912355668605), 'vitb_dino_type-mv': (0.7049741506310939, 0

Plot Configs

In [4]:
title_fontsize = 20
labels_fontsize = 20
ticks_fontsize=16
line_width = 4
spines_width = 2
fig_width = 6
fig_height = 6


Generate Results for fly-anipose dataset
# Finetune 100 frames

In [ ]:
views = ['Cam-A', 'Cam-B', 'Cam-C', 'Cam-D', 'Cam-E', 'Cam-F']
models_to_plot = [
    # 'resnet50_animal_ap10k_type-sv',
    # 'vits_dino_type-sv',
    'vitb_imagenet_type-sv',
    'vitb_dinov3_type-sv',
    'vitb-mae_type-sv',
    'vits_dino_type-mv',
    # 'vitb_imagenet_type-mv',
    # 'vitb-mae_type-mv',
    # 'vits_dino_type-mva',
    # 'vitb_imagenet_type-mva',
    # 'vitb-mae_type-mva',
    'vitl_dinov3_type-mva',
    'vggt_type-mva',
]
seed_list = [0,1,2]
ens = Ensemble()

colors = sns.color_palette("husl", len(models_to_plot))
# Define the color mapping
color_mapping = {k: colors[i] for i, k in enumerate(models_to_plot)}

data_to_plot = {}
for model in models_to_plot:
    for seed in seed_list:
        data_to_plot[f"{model}.{seed}"] = get_litpose_save_dir('fly-anipose', model, 100, seed, 'ft')

ens.data_to_plot = data_to_plot

for key, val in ens.data_to_plot.items():
    for view in views:
        ens.data_to_plot[key] = os.path.join(val, f'predictions_{view}_new.csv')

build_ensemble(ens)      

def major_model(row_data):
    return '.'.join(row_data.split('.')[:-1])
ens.df_line2['model2'] = ens.df_line2['model'].apply(major_model)

df_line2 = ens.df_line2
n_points_dict = ens.n_points_dict
std_vals = ens.std_vals

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_height))

for model in models_to_plot:
    data = df_line2[df_line2['model2'] == model]
    sns.lineplot(x='ens-std', y='pixel_error', data=data, label=model, 
                 color=color_mapping[model], ax=ax, errorbar='se', linewidth=line_width)

ax.set_title('Fly-anipose 100-Frame Finetune', fontsize=title_fontsize)
# ax.set_ylabel('Pixel error', fontsize=labels_fontsize)
ax.set_xlabel('Ensemble std dev', fontsize=labels_fontsize)
ax.set_ylim(bottom=0)  # Ensure y-axis starts at 0
ax.tick_params(axis='both', which='major', labelsize=ticks_fontsize)

# Customize legend
# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)
# Turn off legend
# ax.legend().remove()
ax.set_ylabel('')
# Plot annotations
percentiles = [95, 50, 5]
vals, prctiles = compute_percentiles(
    arr=n_points_dict[models_to_plot[0] + '.0'],
    std_vals=std_vals,
    percentiles=percentiles,
)

for p, v in zip(prctiles, vals):
    ax.axvline(v, ymax=0.95, linestyle='--', linewidth=line_width, color='black', alpha=0.5)
    ax.text(v, ax.get_ylim()[1], f'{p}%', ha='left', va='top', fontsize=labels_fontsize, rotation=90)

# set spines width
for spine in ax.spines.values():
    spine.set_linewidth(spines_width)

# set right and top spines invisible
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.tight_layout()
# plt.grid(True, alpha=0.3)
plt.show()


Generate Results for mirror-mouse dataset

This is the results of 100-frame mirror-mouse:
Dataset: mirror-mouse, Mode: Fine-Tune, Train Frame: 100

In [ ]:
# views of mirror-mouse-separate
dataset = 'mirror-mouse-separate'
views = ['top', 'bot']
# litpose_save_dir = f'{LITPOSE_SAVE_DIR}/{dataset}'
ens = Ensemble()

models_to_plot = [
    # 'resnet50_animal_ap10k_type-sv',
    # 'vits_dino_type-sv',
    'vitb_imagenet_type-sv',
    'vitb_dinov3_type-sv',
    'vitb-mae_type-sv',
    'vits_dino_type-mv',
    # 'vitb_imagenet_type-mv',
    # 'vitb-mae_type-mv',
    # 'vits_dino_type-mva',
    # 'vitb_imagenet_type-mva',
    # 'vitb-mae_type-mva',
    'vitl_dinov3_type-mva',
    'vggt_type-mva',
]

colors = sns.color_palette("husl", len(models_to_plot))
# Define the color mapping
color_mapping = {k: colors[i] for i, k in enumerate(models_to_plot)}
seed_list = [0,1,2]
data_to_plot = {}
for model in models_to_plot:
    for seed in seed_list:
        data_to_plot[f"{model}.{seed}"] = get_litpose_save_dir(dataset, model, 100, seed, 'ft')

ens.data_to_plot = data_to_plot

for key, val in ens.data_to_plot.items():
    for view in views:
        ens.data_to_plot[key] = os.path.join(val, f'predictions_{view}_new.csv')

build_ensemble(ens)      

def major_model(row_data):
    return '.'.join(row_data.split('.')[:-1])
ens.df_line2['model2'] = ens.df_line2['model'].apply(major_model)

df_line2 = ens.df_line2
n_points_dict = ens.n_points_dict
std_vals = ens.std_vals

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_height))

for model in models_to_plot:
    data = df_line2[df_line2['model2'] == model]
    sns.lineplot(x='ens-std', y='pixel_error', data=data, label=model, 
                 color=color_mapping[model], ax=ax, errorbar='se', linewidth=line_width)

ax.set_title(f'{dataset} 100-Frame Finetune', fontsize=title_fontsize)
# ax.set_ylabel('Pixel error', fontsize=labels_fontsize)
ax.set_xlabel('Ensemble std dev', fontsize=labels_fontsize)
ax.set_ylim(bottom=0)  # Ensure y-axis starts at 0
ax.tick_params(axis='both', which='major', labelsize=ticks_fontsize)

# Customize legend
# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)
# Turn off legend
# ax.legend().remove()
ax.set_ylabel('')
# Plot annotations
percentiles = [95, 50, 5]
vals, prctiles = compute_percentiles(
    arr=n_points_dict[models_to_plot[0] + '.0'],
    std_vals=std_vals,
    percentiles=percentiles,
)

for p, v in zip(prctiles, vals):
    ax.axvline(v, ymax=0.95, linestyle='--', linewidth=line_width, color='black', alpha=0.5)
    ax.text(v, ax.get_ylim()[1], f'{p}%', ha='left', va='top', fontsize=labels_fontsize, rotation=90)

# set spines width
for spine in ax.spines.values():
    spine.set_linewidth(spines_width)

# set right and top spines invisible
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.tight_layout()
# plt.grid(True, alpha=0.3)
plt.show()


Chickadee Dataset Results

No NaN values detected in preds array.
Total pixel error for model vggt_type-mva.0: 17103.677978220923
Total pixel error for model vggt_type-mva.1: 18590.481105108604
Total pixel error for model vggt_type-mva.2: 18093.856206517252


In [5]:
dataset = 'chickadee'
views = ['lBack', 'rBack', 'lFront', 'rFront', 'lTop', 'rTop']
models_to_plot = [
    # 'resnet50_animal_ap10k_type-sv',
    # 'vits_dino_type-sv',
    # 'vitb_imagenet_type-sv',
    # 'vitb_dinov3_type-sv',
    # 'vitb-mae_type-sv',
    # 'vits_dino_type-mv',
    # 'vitb_imagenet_type-mv',
    # 'vitb-mae_type-mv',
    # 'vits_dino_type-mva',
    # 'vitb_imagenet_type-mva',
    # 'vitb-mae_type-mva',
    # 'vitl_dinov3_type-mva',
    'vggt_type-mva',
]

colors = sns.color_palette("husl", len(models_to_plot))
# Define the color mapping
color_mapping = {k: colors[i] for i, k in enumerate(models_to_plot)}

seed_list = [0,1,2]
ens = Ensemble()

data_to_plot = {}
for model in models_to_plot:
    for seed in seed_list:
        data_to_plot[f"{model}.{seed}"] = get_litpose_save_dir(dataset, model, 100, seed, 'ft')

ens.data_to_plot = data_to_plot

for key, val in ens.data_to_plot.items():
    for view in views:
        ens.data_to_plot[key] = os.path.join(val, f'predictions_{view}_new.csv')

build_ensemble(ens)      

def major_model(row_data):
    return '.'.join(row_data.split('.')[:-1])
ens.df_line2['model2'] = ens.df_line2['model'].apply(major_model)

df_line2 = ens.df_line2
n_points_dict = ens.n_points_dict
std_vals = ens.std_vals

fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_height))

for model in models_to_plot:
    data = df_line2[df_line2['model2'] == model]
    sns.lineplot(x='ens-std', y='pixel_error', data=data, label=model, 
                 color=color_mapping[model], ax=ax, errorbar='se', linewidth=line_width)

ax.set_title(f'{dataset} 100-Frame Finetune', fontsize=title_fontsize)
# ax.set_ylabel('Pixel error', fontsize=labels_fontsize)
ax.set_xlabel('Ensemble std dev', fontsize=labels_fontsize)
ax.set_ylim(bottom=0)  # Ensure y-axis starts at 0
ax.tick_params(axis='both', which='major', labelsize=ticks_fontsize)

# Customize legend
# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)
# Turn off legend
# ax.legend().remove()
ax.set_ylabel('')
# Plot annotations
percentiles = [95, 50, 5]
vals, prctiles = compute_percentiles(
    arr=n_points_dict[models_to_plot[0] + '.0'],
    std_vals=std_vals,
    percentiles=percentiles,
)

for p, v in zip(prctiles, vals):
    ax.axvline(v, ymax=0.95, linestyle='--', linewidth=line_width, color='black', alpha=0.5)
    ax.text(v, ax.get_ylim()[1], f'{p}%', ha='left', va='top', fontsize=labels_fontsize, rotation=90)

# set spines width
for spine in ax.spines.values():
    spine.set_linewidth(spines_width)

# set right and top spines invisible
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.tight_layout()
# plt.grid(True, alpha=0.3)
plt.show()
